In [216]:
%run utilities
%run noninteractive_utilities

import pandas as pd

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})
dpi = 300
mpl.rcParams['figure.dpi']= dpi

state = "MD"

filename = "data/chapter-0-ensemble-data/" + state + "/rundata_PRES16.csv"

seat_shares_and_vote_shares = (pd.read_csv(filename)).values
seat_totals = seat_shares_and_vote_shares[:,1]
vote_shares_by_district = 1 - seat_shares_and_vote_shares[:,2:]

democrat_seat_shares = seat_totals/np.shape(vote_shares_by_district)[1]
republican_seat_shares = 1 - democrat_seat_shares


In [ ]:
bulk_votes = []
bulk_seats = []
actual_vote_shares = []
actual_seat_shares = []

all_mean_medians = []
all_beta_halves = []

for i in range(np.shape(republican_seat_shares)[0]):
    actual_vote_share = np.mean(vote_shares_by_district[i, :])
    actual_seat_share = republican_seat_shares[i]
    votes, seats = ups_linear(vote_shares_by_district[i, :])
    
    actual_seat_shares.append(actual_seat_share)
    actual_vote_shares.append(actual_vote_share)
    
    bulk_votes.append(votes)
    bulk_seats.append(seats)
    
    mean_median = 0.5 - compute_mean_median_intercept(votes, seats)
    beta_half = compute_partisan_bias_intercept(votes, seats) - 0.5
    all_mean_medians.append(mean_median)
    all_beta_halves.append(beta_half)

    if i % 1000 == 0:
        print(str(100*i/(np.shape(republican_seat_shares)[0])) + "%... ",end='')
    


0.0%... 2.0%... 4.0%... 6.0%... 8.0%... 10.0%... 12.0%... 14.0%... 16.0%... 18.0%... 20.0%... 22.0%... 24.0%... 26.0%... 28.0%... 30.0%... 32.0%... 34.0%... 36.0%... 38.0%... 40.0%... 42.0%... 44.0%... 46.0%... 48.0%... 50.0%... 52.0%... 54.0%... 56.0%... 58.0%... 60.0%... 62.0%... 64.0%... 66.0%... 68.0%... 70.0%... 72.0%... 74.0%... 76.0%... 78.0%... 80.0%... 82.0%... 84.0%... 86.0%... 88.0%... 90.0%... 92.0%... 94.0%... 

In [ ]:
number_of_curves_to_plot = 1000
reordering = np.random.permutation(len(actual_vote_shares))
inds = (reordering[:number_of_curves_to_plot]).tolist()

actual_vote_shares_subset = [actual_vote_shares[i] for i in inds]
actual_seat_shares_subset = [actual_seat_shares[i] for i in inds]
votes_subset = [bulk_votes[i] for i in inds]
seats_subset = [bulk_seats[i] for i in inds]


plot_bulk_ups(actual_vote_shares_subset, actual_seat_shares_subset, votes_subset, seats_subset, text=state,
                  x_label="Average district Republican vote share")

In [ ]:
def plot_ensemble_histogram(x, x_label,y_label,seats=False):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    font_name = 'Helvetica'
    font_size = 15
    if seats:
        bins = np.arange(min(x)-1.25,max(x) + 1.5,0.5)
        ax.hist(x,bins=bins,color=[0.8,0.2,0.5,0.8])
        ax.set_xticks(range(int(min(x)),int(max(x)+1)))
    else:
        ax.hist(x,100,color=[0.8,0.2,0.5,0.8])

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    ax.set_xlabel(x_label, fontname=font_name, fontsize=font_size)
    ax.set_ylabel(y_label, fontname=font_name, fontsize=font_size)


plot_ensemble_histogram(all_mean_medians, "Mean-median", "Counts")
plot_ensemble_histogram(all_beta_halves,"$\\beta(0.5)$", "Counts")
plot_ensemble_histogram(seat_totals,"Seats for Democrats", "Counts",seats=True)

In [ ]:
all_deltas = []

for i in range(np.shape(vote_shares_by_district)[0]):
    republican_vote_shares_by_district = vote_shares_by_district[i,:]
    republican_won_districts = republican_vote_shares_by_district[republican_vote_shares_by_district >= 0.5]
    democrat_won_districts = republican_vote_shares_by_district[republican_vote_shares_by_district < 0.5]
    average_republican_margin = np.mean(republican_won_districts - 0.5)
    average_democrat_margin = np.mean(0.5 - democrat_won_districts)
    
    delta = average_democrat_margin - average_republican_margin
    all_deltas.append(delta)
    

plot_ensemble_histogram(all_deltas, "$d$", "Counts",)



In [ ]:
all_deltas = np.array(all_deltas)
np.mean(all_deltas)